In [3]:
import pandas
import pandasql
cleaned_air=pandas.read_csv('cleaned_air.csv')
cleaned_air[:5]

,Operating Airline,Operating Airline IATA Code,Published Airline,Published Airline IATA Code,GEO Summary,GEO Region,Landing Aircraft Type,Aircraft Body Type,Aircraft Manufacturer,Aircraft Model,Landing Count,Total Landed Weight
0,SkyWest Airlines,OO,United Airlines,UA,International,Canada,Passenger,Regional Jet,Bombardier,CRJ2,30,1410000
1,Air Canada,AC,Air Canada,AC,International,Canada,Passenger,Narrow Body,Airbus,A320,125,17787978
2,Japan Airlines,JL,Japan Airlines,JL,International,Asia,Passenger,Wide Body,Boeing,B773,30,16620000
3,"COPA Airlines, Inc.",CM,"COPA Airlines, Inc.",CM,International,Central America,Passenger,Narrow Body,Boeing,B739,3,491700
4,Hong Kong Airlines Limited,HX,Hong Kong Airlines Limited,HX,International,Asia,Passenger,Wide Body,Airbus,A359,16,7301712


In [4]:
##Estandarización de aerolíneas
###Contar cantidad de aerolíneas operativas y publicadas
print('Aerolíneas Publicadas:')
len(cleaned_air['Published Airline'].drop_duplicates())

Aerolíneas Publicadas:


97

In [5]:
print('Aerolíneas Operativas:')
len(cleaned_air['Operating Airline'].drop_duplicates())

Aerolíneas Operativas:


107

In [6]:
###Búsqueda de datos anormales con una estructura parecida a - Pre 07/01/2013
anormal_data_published=cleaned_air[cleaned_air['Published Airline'].str.contains(r'.-\s\w+\s\d')].groupby('Published Airline')['Published Airline'].count()
anormal_data_published=anormal_data_published.to_frame()
anormal_data_published.description='Anormal data from published airlines'
anormal_data_published

,Published Airline
Published Airline,
United Airlines - Pre 07/01/2013,2274


In [7]:
###Búsqueda de datos anormales con una estructura parecida a - Pre 07/01/2013
anormal_data_operating=cleaned_air[cleaned_air['Operating Airline'].str.contains(r'.-\s\w+\s\d')].groupby('Operating Airline')['Operating Airline'].count()
anormal_data_operating=anormal_data_operating.to_frame()
anormal_data_operating.description='Anormal data from operating airlines'
anormal_data_operating

,Operating Airline
Operating Airline,
United Airlines - Pre 07/01/2013,1834


In [8]:
##Reemplazo de datos
import re
def replace_anormal_airlines(anormal_data,tipo_aerolinea):
    ##Muestra en qué columna se reemplazaron los datos, qué dataframe de datos anormales fue utilizado para el reemplazo y por cuáles datos se reemplazaron
    print('Replaced data in '+tipo_aerolinea+' with the dataset: '+anormal_data.description)
    print('--------------------------------------------------------------------')
    ##Estructura for que itera encima del dataframe de datos anormales y obtiene el string de cada fila
    for anormal_airline in anormal_data.index:
        ##Recoge la parte anormal del string
        anormal_string=re.findall(r'.-\s\w+\s\d.+',anormal_airline)
        ##Separa el string original justo en donde se encuentra la parte anormal
        splitted_string=anormal_airline.split(anormal_string[0])
        ##Recoge la parte del string que se ajusta a la estandarización
        structured_string=splitted_string[0]
        ##Reemplaza los datos anormales en la columna indicada del dataframe de aerolíneas con el string estandarizado
        cleaned_air[tipo_aerolinea]=cleaned_air[tipo_aerolinea].replace(anormal_airline,structured_string)
        print(structured_string)
    print('--------------------------------------------------------------------')
replace_anormal_airlines(anormal_data_operating,'Operating Airline')
replace_anormal_airlines(anormal_data_published,'Published Airline')

Replaced data in Operating Airline with the dataset: Anormal data from operating airlines
--------------------------------------------------------------------
United Airlines
--------------------------------------------------------------------
Replaced data in Published Airline with the dataset: Anormal data from published airlines
--------------------------------------------------------------------
United Airlines
--------------------------------------------------------------------


In [7]:
##Asegurar que se hayan limpiado los datos
cleaned_air[cleaned_air['Operating Airline'].str.contains(r'.-\s\w+\s\d')].groupby('Operating Airline')['Operating Airline'].count()
cleaned_air[cleaned_air['Published Airline'].str.contains(r'.-\s\w+\s\d')].groupby('Published Airline')['Published Airline'].count()

Series([], Name: Published Airline, dtype: int64)

In [8]:
##Exportar datos estandarizados
cleaned_air.to_csv('cleaned_airlines.csv')